In [1]:
import pickle

import pandas as pd
import weave
from IPython.display import display, Markdown

from ydnpd.generation.agent.specifications import *
from ydnpd.generation.agent.utils import produce_datasets
from ydnpd.generation.config import LLM_PATHS
from ydnpd.datasets import load_dataset, mix_datasets, mix_datasets, plot_distribution_distances, plot_dataset_selection, compare_datasets
from ydnpd.utils import metadata_to_pandera_schema
from ydnpd.harness import ALL_EXPERIMENTS

weave.init("ydnpd-data_gen_agent")

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


weave version 0.51.28 is available!  To upgrade, please run:
 $ pip install weave --upgrade
Logged in as Weights & Biases user: shlomihod.
View Weave data at https://wandb.ai/shlomihod/ydnpd-data_gen_agent/weave


In [2]:
PRIVATE_DATASET = "acs/national"
FAMILY, _ = PRIVATE_DATASET.split("/")
NUM_SAMPLES = len(load_dataset(PRIVATE_DATASET)[0])
METADATA_PANDERA = metadata_to_pandera_schema(load_dataset(PRIVATE_DATASET)[1])

NUM_DATASETS = 2
SPECIFICATIONS = [SPECIFICATION_V1] #  SPECIFICATION_V0, SPECIFICATION_V01, SPECIFICATION_V02, SPECIFICATION_V03,

In [ ]:
results = []

for specificaion_version, specification in enumerate(SPECIFICATIONS):

    for llm_path in LLM_PATHS:
        
        if specificaion_version == 1 and "o1-" in llm_path:
            continue

        llm_kwargs = {"llm_path": llm_path,
                        "llm_temperature": llm_temperature,
                        "verbose": False}

        produced_datasets, *_ = produce_datasets(PRIVATE_DATASET,
                                            specification,
                                            NUM_SAMPLES,
                                            NUM_DATASETS,
                                            **llm_kwargs)
        for dataset in produced_datasets:
            results.append({"llm_path": llm_path,
                            "specificaion_version": specificaion_version,
                            "dataset": dataset})

0
🍩 https://wandb.ai/shlomihod/ydnpd-data_gen_agent/r/call/01946b76-ecf2-7c53-895a-6e8f5b21762c
🍩 https://wandb.ai/shlomihod/ydnpd-data_gen_agent/r/call/01946b77-2066-7241-8993-6eb5dcf5d468
🍩 https://wandb.ai/shlomihod/ydnpd-data_gen_agent/r/call/01946b77-48dd-7db0-b1c7-ed1cbf78bc8f
🍩 https://wandb.ai/shlomihod/ydnpd-data_gen_agent/r/call/01946b77-7b17-7f90-872b-f4b4c4aa659a
🍩 https://wandb.ai/shlomihod/ydnpd-data_gen_agent/r/call/01946b77-a412-77f3-98ea-c3adc54e61da
🍩 https://wandb.ai/shlomihod/ydnpd-data_gen_agent/r/call/01946b77-cd8e-71a0-994b-7c68fdbe5706
🍩 https://wandb.ai/shlomihod/ydnpd-data_gen_agent/r/call/01946b77-cd90-70f1-9e96-7fe1aac3b923
🍩 https://wandb.ai/shlomihod/ydnpd-data_gen_agent/r/call/01946b78-09dd-7aa2-a4a5-cc075491c440


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


🍩 https://wandb.ai/shlomihod/ydnpd-data_gen_agent/r/call/01946b78-88c0-79a0-b102-1c499c142706


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


🍩 https://wandb.ai/shlomihod/ydnpd-data_gen_agent/r/call/01946b78-e895-73d3-bc04-bbd1c9ae9f05


...

{'SEX': tensor(0.), 'RAC1P': tensor(0), 'EDU': tensor(0), 'MSP': tensor(0), 'PINCP_DECILE': tensor(3), 'OWN_RENT': tensor(0.), 'HOUSING_TYPE': tensor(2)}
🍩 https://wandb.ai/shlomihod/ydnpd-data_gen_agent/r/call/01946b86-58e1-7a60-b449-cde49ecec6c8
{'SEX': tensor(1.), 'RAC1P': tensor(8), 'EDU': tensor(1), 'MSP': tensor(2), 'PINCP_DECILE': tensor(0), 'OWN_RENT': tensor(1.), 'HOUSING_TYPE': tensor(2)}


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


{'SEX': tensor(1.), 'RAC1P': tensor(4), 'EDU': tensor(6), 'MSP': tensor(1), 'PINCP_DECILE': tensor(1), 'OWN_RENT': tensor(2.), 'HOUSING_TYPE': tensor(1)}
🍩 https://wandb.ai/shlomihod/ydnpd-data_gen_agent/r/call/01946b86-f92c-74b3-b944-a05bf9ed9e03
{'SEX': tensor(1, dtype=torch.int32), 'RAC1P': tensor(1, dtype=torch.int32), 'EDU': tensor(3, dtype=torch.int32), 'MSP': tensor(2, dtype=torch.int32), 'PINCP_DECILE': tensor(1, dtype=torch.int32), 'OWN_RENT': tensor(2, dtype=torch.int32), 'HOUSING_TYPE': tensor(2, dtype=torch.int32)}


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


{'SEX': tensor(1, dtype=torch.int32), 'RAC1P': tensor(1, dtype=torch.int32), 'EDU': tensor(7, dtype=torch.int32), 'MSP': tensor(4, dtype=torch.int32), 'PINCP_DECILE': tensor(0, dtype=torch.int32), 'OWN_RENT': tensor(2, dtype=torch.int32), 'HOUSING_TYPE': tensor(2, dtype=torch.int32)}
🍩 https://wandb.ai/shlomihod/ydnpd-data_gen_agent/r/call/01946b87-a580-76e3-911a-34262342cf16


In [ ]:
results

In [ ]:
# with open("llm-datasets.pkl", "wb") as f:
#     pickle.dump(results, f)

# with open("llm-datasets.pkl", "rb") as f:
#     results = pickle.load(f)

In [ ]:
llm_datasets = {}

llm_gorupby = pd.DataFrame(results).groupby(["llm_name", "llm_temperature", "specificaion_version"])

for key, group in llm_gorupby:
    name = "-".join(map(str, key))

    display(Markdown(f"## {name}"))

    group_llm_datasets = {}

    for i, (_, row) in enumerate(group.iterrows()):
        group_llm_datasets[f"{name}-{i}"] = row["dataset"]
    
    fig, _ = plot_dataset_selection(list(group_llm_datasets.values()))
    display(fig)

    llm_datasets |= group_llm_datasets

plot_dataset_selection(list(llm_datasets.values()));

In [6]:
MIXTURE_MAX_COV_OVERALL_NUM_DATASETS = 10
MIXTURE_MAX_COV_PER_GROUP_NUM_DATASETS = 5

mixture_llm_datasets = {
    "MIX-UNIF": mix_datasets(list(llm_datasets.values()), NUM_SAMPLES, "uniform"),
    "MIX-MAX": mix_datasets(list(llm_datasets.values()), NUM_SAMPLES, "max_coverage", MIXTURE_MAX_COV_OVERALL_NUM_DATASETS)

}

for key, group in llm_gorupby:
    name = "-".join(map(str, key))

    mixture_llm_datasets[f"{name}-MIX-UNIF"] = mix_datasets(list(group["dataset"]), NUM_SAMPLES, "uniform")
    mixture_llm_datasets[f"{name}-MIX-MAX"] = mix_datasets(list(group["dataset"]), NUM_SAMPLES, "max_coverage", MIXTURE_MAX_COV_PER_GROUP_NUM_DATASETS)


llm_datasets |= mixture_llm_datasets

In [ ]:
datasets = {
    name.split("/")[1]: load_dataset(name)[0]
    for name in ALL_EXPERIMENTS[FAMILY].dev_names
} | llm_datasets

plot_distribution_distances(datasets)

In [10]:
for name, df in datasets.items():
    if "MIX" in name:
        df.to_csv(f"llm_datasets/{FAMILY}/{name.replace('.','_')}.csv")